In [1]:
# Get raw data
with open('input/16.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
import numpy as np

In [3]:
# Part 1
signal = np.array(list(rawinput)).astype(int)
slen = signal.shape[0]

mult = np.stack([np.tile(np.repeat([0,1,0,-1], i+1),
                         np.ceil((slen+1) / 4 / (i+1)).astype(int))[1:slen+1]
                 for i in range(slen)])

for i in range(100):
    signal = np.mod(np.abs(np.sum(signal * mult, axis=1)), 10)

np.sum(signal[:8] * (10**np.arange(7,-1,-1))).item()

17978331

In [ ]:
# Part 2

In [316]:
# # Scaled down example - Signal = 8 digits * 1000 - need to find a faster way to replicate sigxmult
# signal = np.array([1,2,3,4,5,6,7,8]*1000).astype(int)
# slen = signal.shape[0]

# mult = np.stack([np.tile(np.repeat([0,1,0,-1], i+1),
#                          np.ceil((slen+1) / 4 / (i+1)).astype(int))[1:slen+1]
#                  for i in range(slen)])

# sigxmult = np.sum(signal * mult, axis=1)

# # for i in range(100):
# # new_signal = np.mod(np.abs(np.sum(signal * mult, axis=1)), 10)
# # new_signal[:50]
# # np.sum(signal[:8] * (10**np.arange(7,-1,-1))).item()

# sigxmult[:50]

In [318]:
signal = np.array(list(rawinput)).astype(int)
slen = signal.shape[0]
stot = np.sum(signal)
n_rept = 100
slen_full = slen * n_rept

slen, stot, slen_full

(650, np.int64(2825), 65000)

In [319]:
pd_patt = []
pd_sigxpatt = []
pd_inc = []
pd_tot = []
for j in range(slen):
    pd_patt += [(z:=np.tile(np.roll(np.repeat([0,1,0,-1], j+1), -1), slen))]
    pd_sigxpatt += [z * np.tile(signal, (j+1)*4)]
    pd_tot += [(z:=np.cumsum(np.tile(np.roll(np.repeat([0,1,0,-1], j+1), 
                                             -1), 
                                     slen) 
                             * np.tile(signal, (j+1)*4)))[-1]]
    pd_inc += [z]
pd_inc = np.array([([0]+i.tolist()+[-1]*(z:=len(pd_inc[-1])))[:z] for i in pd_inc])
pd_tot = np.stack(pd_tot)

KeyboardInterrupt: 

In [320]:
j

282

In [ ]:
rng0 = np.arange(slen_full)
rng1 = rng0+1
rsig_len = rng1*slen*4
rsig_gp = np.mod(rng0,slen)
part_len = np.mod(slen_full, rsig_len)
lbix = np.minimum(((x:=np.expand_dims(part_len, 1))+1) 
                  // ((y:=np.expand_dims(rng1, 1)) * 4) * y * 4
                  + y * (z:=np.expand_dims(np.arange(1,5), 0))
                  + np.expand_dims(rsig_gp+1, 1) * np.mod(z, 2) - 1,
                  x)
lb_pos = (np.sum(np.where(slen-r <= (y:=np.mod(-lbix[:, 0:1], slen)), s, 0), axis=1)
          + np.sum(np.where(r < (z:=np.mod(lbix[:, 1:2], slen)), s, 0), axis=1)
          + ((np.ptp(lbix[:, :2], axis=1)-np.squeeze(y+z))//slen * stot))
lb_neg = (np.sum(np.where(slen-r <= (y:=np.mod(-lbix[:, 2:3], slen)), s, 0), axis=1)
          + np.sum(np.where(r < (z:=np.mod(lbix[:, 3:4], slen)), s, 0), axis=1)
          + ((np.ptp(lbix[:, 2:], axis=1)-np.squeeze(y+z))//slen * stot))

totp1 = slen_full // rsig_len * pd_tot[rsig_gp]
totp2 = pd_inc[rsig_gp, 
               ((part_len+1) // rng1 * (z:=np.mod(rng0, slen)+1)) 
               + np.minimum(np.mod(part_len+1, rng1), z) - 1]
totp3 = lb_pos - lb_neg


In [148]:
# for k in pd_sigxpatt[16].reshape(-1,slen):
#     print(','.join(k.astype(str).tolist()))